# Mlb Player_info

## imports 

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import timedelta
import os
import sys

In [3]:
db_path = '/Users/daniellarson/Desktop/Code/Projects/dodgers_injtrkr/mlb/mlb_players.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Set up your DB path
base_dir = '/Users/daniellarson/Desktop/Code/Projects/dodgers_injtrkr'
db_path = os.path.join(base_dir, 'data', 'mlb_players.db')  
pd.set_option('display.max_columns', None)

In [4]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn) 
print(tables) 

               name
0   mlb_player_info
1   sqlite_sequence
2      injury_stats
3  mlb_player_stats


## MLB Player info 

In [12]:
mlb_player_info = pd.read_sql_query("SELECT * FROM mlb_player_info", conn)

mlb_player_info.head()


,id,mlb_player_id,fullName,firstName,lastName,birthDate,currentAge,birthCity,birthStateProvince,birthCountry,height,weight,primaryPosition,batSide,pitchHand,debutDate,active
0,1,671096,Andrew Abbott,Andrew,Abbott,1999-06-01,25,Lynchburg,VA,USA,"6' 0""",192,Pitcher,Left,Left,2023-06-05,1
1,2,682928,CJ Abrams,Paul,Abrams,2000-10-03,24,Alpharetta,GA,USA,"6' 0""",191,Shortstop,Left,Right,2022-04-08,1
2,3,650556,Bryan Abreu,Bryan,Abreu,1997-04-22,28,Santo Domingo Centro,None,Dominican Republic,"6' 1""",230,Pitcher,Right,Right,2019-07-31,1
3,4,677800,Wilyer Abreu,Wilyer,Abreu,1999-06-24,25,Maracaibo,None,Venezuela,"5' 10""",215,Outfielder,Left,Left,2023-08-22,1
4,5,682668,Luisangel Acuña,Luisangel,Acuña,2002-03-12,23,Caracas,None,Venezuela,"5' 8""",181,Second Base,Right,Right,2024-09-14,1


In [ ]:

# Get the last 10 mlb_player_ids

query = """
SELECT mlb_player_id
FROM mlb_player_info
ORDER BY mlb_player_id DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)

,mlb_player_id
0,820862
1,810938
2,808982
3,808975
4,808967
5,808963
6,807713
7,807712
8,806188
9,806185


## MLB Player stats 

In [6]:
mlb_player_stats = pd.read_sql_query("SELECT * FROM mlb_player_stats", conn)

mlb_player_stats

,mlb_player_id,stat_type,stat_group,season,game_date,team_id,team_name,opponent_id,opponent_name,position,gamesPlayed,games,gamesStarted,assists,putOuts,errors,chances,fielding,doublePlays,triplePlays,throwingErrors,rangeFactorPerGame,rangeFactorPer9Inn,innings,inningsPitched,catcherERA,flyOuts,groundOuts,airOuts,passedBall,wins,losses,wildPitches,pickoffs,runs,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,avg,atBats,obp,slg,ops,caughtStealing,stolenBases,stolenBasePercentage,groundIntoDoublePlay,groundIntoTriplePlay,numberOfPitches,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,babip,groundOutsToAirouts,catchersInterference,atBatsPerHomeRun,summary,isHome,isWin,game_id,game_number,day_night,era,game
0,445276,season,fielding,2024,None,111.0,Boston Red Sox,NaN,None,NA,54,54,0.0,1.0,1.0,0.0,2.0,1.000,0.0,0.0,0.0,0.04,0.33,54.2,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na
1,445276,season,hitting,2024,None,111.0,Boston Red Sox,NaN,None,None,54,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,0.0,0.0,NaN,None,None,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.000,0.0,.000,.000,.000,0.0,0.0,.---,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.---,-.--,0.0,-.--,None,NaN,NaN,NaN,NaN,None,na,na
2,455117,season,fielding,2024,None,145.0,Chicago White Sox,NaN,None,NA,48,48,46.0,11.0,378.0,2.0,391.0,.995,0.0,0.0,1.0,8.10,9.02,388.2,None,4.61,NaN,NaN,NaN,3.0,None,None,18.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,3.0,39.0,.929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,1.0,None,None,NaN,NaN,NaN,NaN,None,na,na
3,455117,season,hitting,2024,None,145.0,Chicago White Sox,NaN,None,None,48,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,30.0,41.0,NaN,None,None,NaN,NaN,9.0,3.0,0.0,4.0,51.0,8.0,0.0,16.0,1.0,.119,135.0,.174,.230,.404,0.0,0.0,.---,5.0,NaN,556.0,147.0,31.0,11.0,56.0,3.0,0.0,.150,0.73,0.0,33.75,None,NaN,NaN,NaN,NaN,None,na,na
4,455117,career,fielding,None,None,NaN,None,NaN,None,NA,1132,1132,1053.0,513.0,9044.0,63.0,9620.0,.993,47.0,0.0,51.0,8.44,9.32,9227.1,None,3.81,NaN,NaN,NaN,78.0,None,None,367.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,184.0,444.0,.707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,16.0,None,None,NaN,NaN,NaN,NaN,None,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436883,810938,gameLog,fielding,2025,2025-05-13,136.0,Seattle Mariners,147.0,New York Yankees,NA,1,1,1.0,3.0,0.0,0.0,3.0,.964,0.0,0.0,0.0,3.00,2.45,11.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,1.0,1.0,777926.0,1.0,day,na,na
436884,810938,gameLog,fielding,2025,2025-05-14,136.0,Seattle Mariners,147.0,New York Yankees,NA,1,1,1.0,2.0,1.0,0.0,3.0,.966,0.0,0.0,0.0,3.00,3.00,9.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,1.0,0.0,777915.0,1.0,day,na,na
436885,810938,gameLog,fielding,2025,2025-05-16,136.0,Seattle Mariners,135.0,San Diego Padres,NA,1,1,1.0,2.0,0.0,0.0,2.0,.967,0.0,0.0,0.0,2.00,2.00,9.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,0.0,1.0,777896.0,1.0,day,na,na
436886,810938,gameLog,fielding,2025,2025-05-17,136.0,Seattle Mariners,135.0,San Diego Padres,NA,1,1,1.0,1.0,0.0,0.0,1.0,.968,0.0,0.0,0.0,1.00,1.00,9.0,None,None,NaN,NaN,NaN

In [ ]:
# Ecample csv of mlb_player_stats

mlb_player_stats_example = mlb_player_stats.head(100)

#mlb_player_stats_example.to_csv('mlb_player_stats_example.csv', index=False)

## Player info and stats

In [31]:
query = """
SELECT mlb_player_stats.* , mlb_player_info.*
FROM mlb_player_stats
JOIN mlb_player_info ON mlb_player_stats.mlb_player_id = mlb_player_info.mlb_player_id
"""
player_pre = pd.read_sql_query(query, conn)

# Reorder columns: fullName and mlb_player_id first, then the rest
cols = ['fullName', 'mlb_player_id'] + [col for col in player_pre.columns if col not in ['fullName', 'mlb_player_id']]
player_pre = player_pre[cols]

# Rename columns to make them more readable
player_df = player_pre.rename(columns={
    'fullName': 'Name',
    'mlb_player_id': 'PlayerID',
    'stat_type': 'StatType',
    'stat_group': 'StatGroup',
    'season': 'Season',
    'game_date': 'GameDate',
    'team_id': 'TeamID',
    'team_name': 'Team',
    'opponent_id': 'OpponentID',
    'opponent_name': 'Opponent',
    'position': 'Position',
    'gamesPlayed': 'GamesPlayed',
    'games': 'Games',
    'gamesStarted': 'GamesStarted',
    'assists': 'Assists',
    'putOuts': 'PutOuts',
    'errors': 'Errors',
    'chances': 'Chances',
    'fielding': 'Fielding%',
    'doublePlays': 'DoublePlays',
    'triplePlays': 'TriplePlays',
    'throwingErrors': 'ThrowingErrors',
    'rangeFactorPerGame': 'RangeFactor/Game',
    'rangeFactorPer9Inn': 'RangeFactor/9Inn',
    'innings': 'Innings',
    'inningsPitched': 'InningsPitched',
    'catcherERA': 'CatcherERA',
    'flyOuts': 'FlyOuts',
    'groundOuts': 'GroundOuts',
    'airOuts': 'AirOuts',
    'passedBall': 'PassedBall',
    'wins': 'Wins',
    'losses': 'Losses',
    'wildPitches': 'WildPitches',
    'pickoffs': 'Pickoffs',
    'runs': 'Runs',
    'doubles': 'Doubles',
    'triples': 'Triples',
    'homeRuns': 'HomeRuns',
    'strikeOuts': 'Strikeouts',
    'baseOnBalls': 'Walks',
    'intentionalWalks': 'IntentionalWalks',
    'hits': 'Hits',
    'hitByPitch': 'HitByPitch',
    'avg': 'BattingAvg',
    'atBats': 'AtBats',
    'obp': 'OnBase%',
    'slg': 'Slugging%',
    'ops': 'OPS',
    'caughtStealing': 'CaughtStealing',
    'stolenBases': 'StolenBases',
    'stolenBasePercentage': 'Steal%',
    'groundIntoDoublePlay': 'GIDP',
    'groundIntoTriplePlay': 'GITP',
    'numberOfPitches': 'PitchesThrown',
    'plateAppearances': 'PlateAppearances',
    'totalBases': 'TotalBases',
    'rbi': 'RBIs',
    'leftOnBase': 'LOB',
    'sacBunts': 'SacBunts',
    'sacFlies': 'SacFlies',
    'babip': 'BABIP',
    'groundOutsToAirouts': 'GO/AO',
    'catchersInterference': 'CatchersInterference',
    'atBatsPerHomeRun': 'AB/HR',
    'summary': 'Summary',
    'isHome': 'HomeGame',
    'isWin': 'Win',
    'game_id': 'GameID',
    'game_number': 'GameNumber',
    'day_night': 'GameTime',
    'era': 'ERA',
    'game': 'Game',
    'id': 'RowID',
    'firstName': 'FirstName',
    'lastName': 'LastName',
    'birthDate': 'BirthDate',
    'currentAge': 'Age',
    'birthCity': 'BirthCity',
    'birthStateProvince': 'BirthState',
    'birthCountry': 'BirthCountry',
    'height': 'Height',
    'weight': 'Weight',
    'primaryPosition': 'PrimaryPosition',
    'batSide': 'BatSide',
    'pitchHand': 'PitchHand',
    'debutDate': 'DebutDate',
    'active': 'IsActive'
})

# Show the updated DataFrame
player_pre.columns

Index(['fullName', 'mlb_player_id', 'mlb_player_id', 'stat_type', 'stat_group',
       'season', 'game_date', 'team_id', 'team_name', 'opponent_id',
       'opponent_name', 'position', 'gamesPlayed', 'games', 'gamesStarted',
       'assists', 'putOuts', 'errors', 'chances', 'fielding', 'doublePlays',
       'triplePlays', 'throwingErrors', 'rangeFactorPerGame',
       'rangeFactorPer9Inn', 'innings', 'inningsPitched', 'catcherERA',
       'flyOuts', 'groundOuts', 'airOuts', 'passedBall', 'wins', 'losses',
       'wildPitches', 'pickoffs', 'runs', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'avg', 'atBats', 'obp', 'slg', 'ops', 'caughtStealing', 'stolenBases',
       'stolenBasePercentage', 'groundIntoDoublePlay', 'groundIntoTriplePlay',
       'numberOfPitches', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'babip', 'groundOutsToAirouts',
       'catchersInterference', 'at

In [ ]:
player_df.columns

player_stats = player_df

#player_stats.to_csv('player_stats.csv', index=False)

In [37]:
player_stats.head(10)
player_stats.columns

Index(['Name', 'PlayerID', 'PlayerID', 'StatType', 'StatGroup', 'Season',
       'GameDate', 'TeamID', 'Team', 'OpponentID', 'Opponent', 'Position',
       'GamesPlayed', 'Games', 'GamesStarted', 'Assists', 'PutOuts', 'Errors',
       'Chances', 'Fielding%', 'DoublePlays', 'TriplePlays', 'ThrowingErrors',
       'RangeFactor/Game', 'RangeFactor/9Inn', 'Innings', 'InningsPitched',
       'CatcherERA', 'FlyOuts', 'GroundOuts', 'AirOuts', 'PassedBall', 'Wins',
       'Losses', 'WildPitches', 'Pickoffs', 'Runs', 'Doubles', 'Triples',
       'HomeRuns', 'Strikeouts', 'Walks', 'IntentionalWalks', 'Hits',
       'HitByPitch', 'BattingAvg', 'AtBats', 'OnBase%', 'Slugging%', 'OPS',
       'CaughtStealing', 'StolenBases', 'Steal%', 'GIDP', 'GITP',
       'PitchesThrown', 'PlateAppearances', 'TotalBases', 'RBIs', 'LOB',
       'SacBunts', 'SacFlies', 'BABIP', 'GO/AO', 'CatchersInterference',
       'AB/HR', 'Summary', 'HomeGame', 'Win', 'GameID', 'GameNumber',
       'GameTime', 'ERA', 'Game', 'R

In [ ]:

# Save the DataFrame to a CSV file
#player_stats.to_csv('mlb_stats.csv', index=False)

## Injury Stats 

In [12]:
injury_stats = pd.read_sql_query("SELECT * FROM injury_stats", conn)

In [70]:
injury_stats.columns

Index(['mlb_player_id', 'player_name', 'injury_date', 'season',
       'injury_description', 'team_name'],
      dtype='object')

In [20]:
injury_stats

,mlb_player_id,player_name,injury_date,season,injury_description,team_name
0,434378,Justin Verlander,2015-04-08,2015,Detroit Tigers placed RHP Justin Verlander on ...,Detroit Tigers
1,434378,Justin Verlander,2015-06-13,2015,Detroit Tigers activated RHP Justin Verlander ...,Detroit Tigers
2,434378,Justin Verlander,2020-07-27,2020,Houston Astros placed RHP Justin Verlander on ...,Houston Astros
3,434378,Justin Verlander,2020-09-28,2020,Houston Astros activated RHP Justin Verlander ...,Houston Astros
4,434378,Justin Verlander,2021-02-27,2021,Houston Astros placed P Justin Verlander on th...,Houston Astros
...,...,...,...,...,...,...
8608,808982,Jung Hoo Lee,2024-05-13,2024,San Francisco Giants placed OF Jung Hoo Lee on...,San Francisco Giants
8609,808982,Jung Hoo Lee,2024-05-18,2024,San Francisco Giants transferred OF Jung Hoo L...,San Francisco Giants
8610,808982,Jung Hoo Lee,2024-11-04,2024,San Francisco Giants activated CF Jung Hoo Lee...,San Francisco Giants
8611,810938,Ben Williamson,2023-08-23,2023,Modesto Nuts placed 3B Ben Williamson on the 7...,Modesto Nuts


In [15]:
injury_stats_exmple = injury_stats.head(100)

injury_stats_exmple.to_csv('injury_stats_example.csv', index=False)

In [ ]:

# Save injury_stats to CSV

#injury_stats.to_csv('injury_stats.csv', index=False)

## Injury Stats and Player Data 

In [22]:

# Load both tables using the existing connection
injuries = pd.read_sql_query("SELECT * FROM injury_stats", conn)
stats = pd.read_sql_query("SELECT * FROM mlb_player_stats", conn)

# Convert to datetime
injuries['injury_date'] = pd.to_datetime(injuries['injury_date'], errors='coerce')
stats['game_date'] = pd.to_datetime(stats['game_date'], errors='coerce')
stats = stats.dropna(subset=['game_date'])

# Only keep meaningful stat rows (optional)
stats = stats[stats['stat_type'].str.lower().str.contains('gamelog|hitting|pitching', na=False)]

# Build merged DataFrame
rows = []
for _, injury in injuries.iterrows():
    player_id = injury['mlb_player_id']
    injury_date = injury['injury_date']

    window_df = stats[
        (stats['mlb_player_id'] == player_id) &
        (stats['game_date'] >= injury_date - timedelta(days=10)) &
        (stats['game_date'] <= injury_date)
    ].copy()

    if not window_df.empty:
        window_df['injury_date'] = injury_date
        window_df['injury_description'] = injury['injury_description']
        rows.append(window_df)

# Final merged table
injury_player_stats = pd.concat(rows, ignore_index=True)
injury_player_stats = injury_player_stats.rename(columns={
    'mlb_player_id': 'PlayerID',
    'game_date': 'GameDate',
    'stat_type': 'StatType',
    'stat_group': 'StatGroup',
    'team_id': 'TeamID',
    'team_name': 'Team',
    'opponent_id': 'OpponentID',
    'opponent_name': 'Opponent',
    'position': 'Position',
    'gamesPlayed': 'GamesPlayed',
    'games': 'Games',
    'gamesStarted': 'GamesStarted',
    # Add other renaming as needed
})

#injury_player_stats.to_csv("injury_player_stats.csv", index=False)
#print("✅ Merged injury window saved to data/injury_merged_window.csv")

In [21]:
injury_player_stats 

,PlayerID,StatType,StatGroup,season,GameDate,TeamID,Team,OpponentID,Opponent,Position,GamesPlayed,Games,GamesStarted,assists,putOuts,errors,chances,fielding,doublePlays,triplePlays,throwingErrors,rangeFactorPerGame,rangeFactorPer9Inn,innings,inningsPitched,catcherERA,flyOuts,groundOuts,airOuts,passedBall,wins,losses,wildPitches,pickoffs,runs,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,avg,atBats,obp,slg,ops,caughtStealing,stolenBases,stolenBasePercentage,groundIntoDoublePlay,groundIntoTriplePlay,numberOfPitches,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,babip,groundOutsToAirouts,catchersInterference,atBatsPerHomeRun,summary,isHome,isWin,game_id,game_number,day_night,era,game,injury_date,injury_description
0,456781,gameLog,fielding,2021,2021-04-12,137.0,San Francisco Giants,113.0,Cincinnati Reds,NA,1,1,1.0,2.0,1.0,0.0,3.0,1.000,0.0,0.0,0.0,3.00,3.00,9.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,1.0,0.0,634648.0,1.0,day,na,na,2021-04-22,San Francisco Giants placed 2B Donovan Solano ...
1,456781,gameLog,fielding,2021,2021-04-13,137.0,San Francisco Giants,113.0,Cincinnati Reds,NA,1,1,0.0,0.0,1.0,0.0,1.0,1.000,0.0,0.0,0.0,1.00,4.50,2.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,1.0,1.0,634510.0,1.0,day,na,na,2021-04-22,San Francisco Giants placed 2B Donovan Solano ...
2,456781,gameLog,fielding,2021,2021-04-14,137.0,San Francisco Giants,113.0,Cincinnati Reds,NA,1,1,1.0,3.0,2.0,0.0,5.0,1.000,1.0,0.0,0.0,5.00,5.00,9.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,1.0,1.0,634582.0,1.0,day,na,na,2021-04-22,San Francisco Giants placed 2B Donovan Solano ...
3,456781,gameLog,fielding,2021,2021-04-16,137.0,San Francisco Giants,146.0,Miami Marlins,NA,1,1,1.0,2.0,1.0,0.0,3.0,1.000,0.0,0.0,0.0,3.00,3.38,8.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,0.0,0.0,634519.0,1.0,day,na,na,2021-04-22,San Francisco Giants placed 2B Donovan Solano ...
4,456781,gameLog,fielding,2021,2021-04-17,137.0,San Francisco Giants,146.0,Miami Marlins,NA,1,1,0.0,1.0,0.0,0.0,1.0,1.000,0.0,0.0,0.0,1.00,3.00,3.2,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,0.0,0.0,634450.0,1.0,day,na,na,2021-04-22,San Francisco Giants placed 2B Donovan Solano ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15259,808982,gameLog,hitting,2024,2024-05-12,137.0,San Francisco Giants,113.0,Cincinnati Reds,None,1,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,0.0,0.0,0.0,NaN,None,None,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.262,0.0,.310,.331,.641,0.0,0.0,.---,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.---,-.--,0.0,-.--,0-0,1.0,1.0,745345.0,1.0,day,na,na,2024-05-18,San Francisco Giants transferred OF Jung Hoo L...
15260,808982,gameLog,hitting,2024,2024-05-08,137.0,San Francisco Giants,115.0,Colorado Rockies,None,1,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,3.0,1.0,3.0,NaN,None,None,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,.262,5.0,.310,.331,.641,0.0,0.0,.---,0.0,0.0,19.0,5.0,2.0,1.0,2.0,0.0,0.0,.200,0.33,0.0,-.--,"1-5 | 2B, RBI, R",0.0,1.0,746562.0,1.0,day,na,na,2024-05-18,San Francisco Giants tra

## The Dodgers Stats

In [27]:
# Step 1: SQL query to join stats and info for all players
query = """
SELECT mlb_player_stats.*, mlb_player_info.*
FROM mlb_player_stats
JOIN mlb_player_info ON mlb_player_stats.mlb_player_id = mlb_player_info.mlb_player_id
WHERE mlb_player_stats.team_name = 'Los Angeles Dodgers' AND Season = 2025
;
"""

# Step 2: Load the result into a pandas DataFrame
dodgers_player_pre = pd.read_sql_query(query, conn)

# Step 4: Rename columns for readability
dodgers_df = dodgers_player_pre.rename(columns={
    'fullName': 'Name',
    'mlb_player_id': 'PlayerID',
    'stat_type': 'StatType',
    'stat_group': 'StatGroup',
    'season': 'Season',
    'game_date': 'GameDate',
    'team_id': 'TeamID',
    'team_name': 'Team',
    'opponent_id': 'OpponentID',
    'opponent_name': 'Opponent',
    'position': 'Position',
    'gamesPlayed': 'GamesPlayed',
    'games': 'Games',
    'gamesStarted': 'GamesStarted',
    'assists': 'Assists',
    'putOuts': 'PutOuts',
    'errors': 'Errors',
    'chances': 'Chances',
    'fielding': 'Fielding%',
    'doublePlays': 'DoublePlays',
    'triplePlays': 'TriplePlays',
    'throwingErrors': 'ThrowingErrors',
    'rangeFactorPerGame': 'RangeFactor/Game',
    'rangeFactorPer9Inn': 'RangeFactor/9Inn',
    'innings': 'Innings',
    'inningsPitched': 'InningsPitched',
    'catcherERA': 'CatcherERA',
    'flyOuts': 'FlyOuts',
    'groundOuts': 'GroundOuts',
    'airOuts': 'AirOuts',
    'passedBall': 'PassedBall',
    'wins': 'Wins',
    'losses': 'Losses',
    'wildPitches': 'WildPitches',
    'pickoffs': 'Pickoffs',
    'runs': 'Runs',
    'doubles': 'Doubles',
    'triples': 'Triples',
    'homeRuns': 'HomeRuns',
    'strikeOuts': 'Strikeouts',
    'baseOnBalls': 'Walks',
    'intentionalWalks': 'IntentionalWalks',
    'hits': 'Hits',
    'hitByPitch': 'HitByPitch',
    'avg': 'BattingAvg',
    'atBats': 'AtBats',
    'obp': 'OnBase%',
    'slg': 'Slugging%',
    'ops': 'OPS',
    'caughtStealing': 'CaughtStealing',
    'stolenBases': 'StolenBases',
    'stolenBasePercentage': 'Steal%',
    'groundIntoDoublePlay': 'GIDP',
    'groundIntoTriplePlay': 'GITP',
    'numberOfPitches': 'PitchesThrown',
    'plateAppearances': 'PlateAppearances',
    'totalBases': 'TotalBases',
    'rbi': 'RBIs',
    'leftOnBase': 'LOB',
    'sacBunts': 'SacBunts',
    'sacFlies': 'SacFlies',
    'babip': 'BABIP',
    'groundOutsToAirouts': 'GO/AO',
    'catchersInterference': 'CatchersInterference',
    'atBatsPerHomeRun': 'AB/HR',
    'summary': 'Summary',
    'isHome': 'HomeGame',
    'isWin': 'Win',
    'game_id': 'GameID',
    'game_number': 'GameNumber',
    'day_night': 'GameTime',
    'era': 'ERA',
    'game': 'Game',
    'id': 'RowID',
    'firstName': 'FirstName',
    'lastName': 'LastName',
    'birthDate': 'BirthDate',
    'currentAge': 'Age',
    'birthCity': 'BirthCity',
    'birthStateProvince': 'BirthState',
    'birthCountry': 'BirthCountry',
    'height': 'Height',
    'weight': 'Weight',
    'primaryPosition': 'PrimaryPosition',
    'batSide': 'BatSide',
    'pitchHand': 'PitchHand',
    'debutDate': 'DebutDate',
    'active': 'IsActive'
})



In [29]:
dodgers_df.columns

Index(['PlayerID', 'StatType', 'StatGroup', 'Season', 'GameDate', 'TeamID',
       'Team', 'OpponentID', 'Opponent', 'Position', 'GamesPlayed', 'Games',
       'GamesStarted', 'Assists', 'PutOuts', 'Errors', 'Chances', 'Fielding%',
       'DoublePlays', 'TriplePlays', 'ThrowingErrors', 'RangeFactor/Game',
       'RangeFactor/9Inn', 'Innings', 'InningsPitched', 'CatcherERA',
       'FlyOuts', 'GroundOuts', 'AirOuts', 'PassedBall', 'Wins', 'Losses',
       'WildPitches', 'Pickoffs', 'Runs', 'Doubles', 'Triples', 'HomeRuns',
       'Strikeouts', 'Walks', 'IntentionalWalks', 'Hits', 'HitByPitch',
       'BattingAvg', 'AtBats', 'OnBase%', 'Slugging%', 'OPS', 'CaughtStealing',
       'StolenBases', 'Steal%', 'GIDP', 'GITP', 'PitchesThrown',
       'PlateAppearances', 'TotalBases', 'RBIs', 'LOB', 'SacBunts', 'SacFlies',
       'BABIP', 'GO/AO', 'CatchersInterference', 'AB/HR', 'Summary',
       'HomeGame', 'Win', 'GameID', 'GameNumber', 'GameTime', 'ERA', 'Game',
       'RowID', 'PlayerID', '

In [28]:
dodgers_df

,PlayerID,StatType,StatGroup,Season,GameDate,TeamID,Team,OpponentID,Opponent,Position,GamesPlayed,Games,GamesStarted,Assists,PutOuts,Errors,Chances,Fielding%,DoublePlays,TriplePlays,ThrowingErrors,RangeFactor/Game,RangeFactor/9Inn,Innings,InningsPitched,CatcherERA,FlyOuts,GroundOuts,AirOuts,PassedBall,Wins,Losses,WildPitches,Pickoffs,Runs,Doubles,Triples,HomeRuns,Strikeouts,Walks,IntentionalWalks,Hits,HitByPitch,BattingAvg,AtBats,OnBase%,Slugging%,OPS,CaughtStealing,StolenBases,Steal%,GIDP,GITP,PitchesThrown,PlateAppearances,TotalBases,RBIs,LOB,SacBunts,SacFlies,BABIP,GO/AO,CatchersInterference,AB/HR,Summary,HomeGame,Win,GameID,GameNumber,GameTime,ERA,Game,RowID,PlayerID,Name,FirstName,LastName,BirthDate,Age,BirthCity,BirthState,BirthCountry,Height,Weight,PrimaryPosition,BatSide,PitchHand,DebutDate,IsActive
0,489446,season,fielding,2025,None,119,Los Angeles Dodgers,NaN,None,NA,22,22,0.0,3.0,2.0,0.0,5.0,1.000,0.0,0.0,0.0,0.23,2.50,18.2,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,1087,489446,Kirby Yates,Kirby,Yates,1987-03-25,38,Lihue,HI,USA,"5' 10""",205,Pitcher,Left,Right,2014-06-07,1
1,518692,season,hitting,2025,None,119,Los Angeles Dodgers,NaN,None,None,37,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,25.0,35.0,NaN,None,None,NaN,NaN,27.0,11.0,1.0,9.0,29.0,15.0,2.0,50.0,1.0,.368,136.0,.426,.662,1.088,1.0,0.0,.000,5.0,NaN,570.0,155.0,90.0,34.0,52.0,0.0,3.0,.406,0.71,0.0,15.11,None,NaN,NaN,NaN,NaN,None,na,na,322,518692,Freddie Freeman,Frederick,Freeman,1989-09-12,35,Villa Park,CA,USA,"6' 4""",220,First Base,Left,Right,2010-09-01,1
2,518692,season,fielding,2025,None,119,Los Angeles Dodgers,NaN,None,NA,36,36,36.0,26.0,246.0,1.0,273.0,.996,21.0,0.0,0.0,7.56,8.21,298.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,322,518692,Freddie Freeman,Frederick,Freeman,1989-09-12,35,Villa Park,CA,USA,"6' 4""",220,First Base,Left,Right,2010-09-01,1
3,518692,gameLog,hitting,2025,2025-03-27,119,Los Angeles Dodgers,116.0,Detroit Tigers,None,1,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,3.0,1.0,3.0,NaN,None,None,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.000,4.0,.000,.000,.000,0.0,0.0,.---,0.0,0.0,16.0,4.0,0.0,0.0,0.0,0.0,0.0,.000,0.33,0.0,-.--,0-4,1.0,1.0,778546.0,1.0,day,na,na,322,518692,Freddie Freeman,Frederick,Freeman,1989-09-12,35,Villa Park,CA,USA,"6' 4""",220,First Base,Left,Right,2010-09-01,1
4,518692,gameLog,hitting,2025,2025-03-28,119,Los Angeles Dodgers,116.0,Detroit Tigers,None,1,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,1.0,1.0,2.0,NaN,None,None,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,.125,4.0,.125,.500,.625,0.0,0.0,.---,0.0,0.0,11.0,4.0,4.0,2.0,0.0,0.0,0.0,.000,0.50,0.0,4.00,"1-4 | HR, 2 RBI, R",1.0,1.0,778544.0,1.0,day,na,na,322,518692,Freddie Freeman,Frederick,Freeman,1989-09-12,35,Villa Park,CA,USA,"6' 4""",220,First Base,Left,Right,2010-09-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,808975,gameLog,hitting,2025,2025-05-14,119,Los Angeles Dodgers,133.0,Athletics,None,1,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,0.0,0.0,0.0,NaN,None,None,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,.360,2.0,.360,.480,.840,0.0,0.0,.---,0.0,0.0,8.0,2.0,5.0,1.0,0.0,0.0,0.0,1.000,-.--,0.0,2.00,"2-2 | HR, RBI, R",1.0,1.0,777917.0,1.0,day,na,na,498,808975,Hyeseong Kim,Hyeseong,Kim,1999-01-27,26,Goyang,None,Republic of Korea,"5' 10"

In [106]:
dodgers_2025 = player_df[(player_df['Team'] == 'Los Angeles Dodgers') & (player_df['Season'] == 2025)]

dodgers_2025

,Name,PlayerID,PlayerID,StatType,StatGroup,Season,GameDate,TeamID,Team,OpponentID,Opponent,Position,GamesPlayed,Games,GamesStarted,Assists,PutOuts,Errors,Chances,Fielding%,DoublePlays,TriplePlays,ThrowingErrors,RangeFactor/Game,RangeFactor/9Inn,Innings,InningsPitched,CatcherERA,FlyOuts,GroundOuts,AirOuts,PassedBall,Wins,Losses,WildPitches,Pickoffs,Runs,Doubles,Triples,HomeRuns,Strikeouts,Walks,IntentionalWalks,Hits,HitByPitch,BattingAvg,AtBats,OnBase%,Slugging%,OPS,CaughtStealing,StolenBases,Steal%,GIDP,GITP,PitchesThrown,PlateAppearances,TotalBases,RBIs,LOB,SacBunts,SacFlies,BABIP,GO/AO,CatchersInterference,AB/HR,Summary,HomeGame,Win,GameID,GameNumber,GameTime,ERA,Game,RowID,FirstName,LastName,BirthDate,Age,BirthCity,BirthState,BirthCountry,Height,Weight,PrimaryPosition,BatSide,PitchHand,DebutDate,IsActive


In [90]:
dodgers_2025 

,Name,PlayerID,PlayerID,StatType,StatGroup,Season,GameDate,TeamID,Team,OpponentID,Opponent,Position,GamesPlayed,Games,GamesStarted,Assists,PutOuts,Errors,Chances,Fielding%,DoublePlays,TriplePlays,ThrowingErrors,RangeFactor/Game,RangeFactor/9Inn,Innings,InningsPitched,CatcherERA,FlyOuts,GroundOuts,AirOuts,PassedBall,Wins,Losses,WildPitches,Pickoffs,Runs,Doubles,Triples,HomeRuns,Strikeouts,Walks,IntentionalWalks,Hits,HitByPitch,BattingAvg,AtBats,OnBase%,Slugging%,OPS,CaughtStealing,StolenBases,Steal%,GIDP,GITP,PitchesThrown,PlateAppearances,TotalBases,RBIs,LOB,SacBunts,SacFlies,BABIP,GO/AO,CatchersInterference,AB/HR,Summary,HomeGame,Win,GameID,GameNumber,GameTime,ERA,Game,RowID,FirstName,LastName,BirthDate,Age,BirthCity,BirthState,BirthCountry,Height,Weight,PrimaryPosition,BatSide,PitchHand,DebutDate,IsActive


## Players Game Logs

In [38]:

mlb_player_stats

,mlb_player_id,stat_type,stat_group,season,game_date,team_id,team_name,opponent_id,opponent_name,position,gamesPlayed,games,gamesStarted,assists,putOuts,errors,chances,fielding,doublePlays,triplePlays,throwingErrors,rangeFactorPerGame,rangeFactorPer9Inn,innings,inningsPitched,catcherERA,flyOuts,groundOuts,airOuts,passedBall,wins,losses,wildPitches,pickoffs,runs,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,avg,atBats,obp,slg,ops,caughtStealing,stolenBases,stolenBasePercentage,groundIntoDoublePlay,groundIntoTriplePlay,numberOfPitches,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,babip,groundOutsToAirouts,catchersInterference,atBatsPerHomeRun,summary,isHome,isWin,game_id,game_number,day_night,era,game
0,445276,season,fielding,2024,None,111.0,Boston Red Sox,NaN,None,NA,54,54,0.0,1.0,1.0,0.0,2.0,1.000,0.0,0.0,0.0,0.04,0.33,54.2,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na
1,445276,season,hitting,2024,None,111.0,Boston Red Sox,NaN,None,None,54,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,0.0,0.0,NaN,None,None,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.000,0.0,.000,.000,.000,0.0,0.0,.---,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.---,-.--,0.0,-.--,None,NaN,NaN,NaN,NaN,None,na,na
2,455117,season,fielding,2024,None,145.0,Chicago White Sox,NaN,None,NA,48,48,46.0,11.0,378.0,2.0,391.0,.995,0.0,0.0,1.0,8.10,9.02,388.2,None,4.61,NaN,NaN,NaN,3.0,None,None,18.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,3.0,39.0,.929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,1.0,None,None,NaN,NaN,NaN,NaN,None,na,na
3,455117,season,hitting,2024,None,145.0,Chicago White Sox,NaN,None,None,48,na,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,30.0,41.0,NaN,None,None,NaN,NaN,9.0,3.0,0.0,4.0,51.0,8.0,0.0,16.0,1.0,.119,135.0,.174,.230,.404,0.0,0.0,.---,5.0,NaN,556.0,147.0,31.0,11.0,56.0,3.0,0.0,.150,0.73,0.0,33.75,None,NaN,NaN,NaN,NaN,None,na,na
4,455117,career,fielding,None,None,NaN,None,NaN,None,NA,1132,1132,1053.0,513.0,9044.0,63.0,9620.0,.993,47.0,0.0,51.0,8.44,9.32,9227.1,None,3.81,NaN,NaN,NaN,78.0,None,None,367.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,184.0,444.0,.707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,16.0,None,None,NaN,NaN,NaN,NaN,None,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436883,810938,gameLog,fielding,2025,2025-05-13,136.0,Seattle Mariners,147.0,New York Yankees,NA,1,1,1.0,3.0,0.0,0.0,3.0,.964,0.0,0.0,0.0,3.00,2.45,11.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,1.0,1.0,777926.0,1.0,day,na,na
436884,810938,gameLog,fielding,2025,2025-05-14,136.0,Seattle Mariners,147.0,New York Yankees,NA,1,1,1.0,2.0,1.0,0.0,3.0,.966,0.0,0.0,0.0,3.00,3.00,9.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,1.0,0.0,777915.0,1.0,day,na,na
436885,810938,gameLog,fielding,2025,2025-05-16,136.0,Seattle Mariners,135.0,San Diego Padres,NA,1,1,1.0,2.0,0.0,0.0,2.0,.967,0.0,0.0,0.0,2.00,2.00,9.0,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,0.0,1.0,777896.0,1.0,day,na,na
436886,810938,gameLog,fielding,2025,2025-05-17,136.0,Seattle Mariners,135.0,San Diego Padres,NA,1,1,1.0,1.0,0.0,0.0,1.0,.968,0.0,0.0,0.0,1.00,1.00,9.0,None,None,NaN,NaN,NaN

In [ ]:
game_log_pre = pd.read_sql_query("SELECT * FROM mlb_player_stats WHERE stat_type == 'gameLog' ", conn)

game_log_pre.columns

desired_columns = [
    'Name', 'GameDate', 'Team', 'Opponent', 'AtBats', 'Runs', 'Hits', 'TotalBases',
    'Doubles', 'Triples', 'HomeRuns', 'RBIs', 'Walks', 'IntentionalWalks',
    'Strikeouts', 'StolenBases', 'CaughtStealing', 'BattingAvg', 'OnBase%',
    'Slugging%', 'HitByPitch', 'SacBunts', 'SacFlies'
]


game_log_stats = game_log_pre[desired_columns]

SyntaxError: invalid syntax (4108918011.py, line 6)

## Stats

In [22]:
query = """
SELECT mlb_player_stats.*, mlb_player_info.*, injury_stats.*
FROM mlb_player_stats
JOIN mlb_player_info ON mlb_player_stats.mlb_player_id = mlb_player_info.mlb_player_id
JOIN injury_stats ON mlb_player_stats.mlb_player_id = injury_stats.mlb_player_id
WHERE mlb_player_stats.team_name = 'Los Angeles Dodgers'
  AND mlb_player_stats.season = 2020
;
"""

dodgers_injury_2025 = pd.read_sql_query(query, conn)

dodgers_injury_2025



,mlb_player_id,stat_type,stat_group,season,game_date,team_id,team_name,opponent_id,opponent_name,position,gamesPlayed,games,gamesStarted,assists,putOuts,errors,chances,fielding,doublePlays,triplePlays,throwingErrors,rangeFactorPerGame,rangeFactorPer9Inn,innings,inningsPitched,catcherERA,flyOuts,groundOuts,airOuts,passedBall,wins,losses,wildPitches,pickoffs,runs,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,avg,atBats,obp,slg,ops,caughtStealing,stolenBases,stolenBasePercentage,groundIntoDoublePlay,groundIntoTriplePlay,numberOfPitches,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,babip,groundOutsToAirouts,catchersInterference,atBatsPerHomeRun,summary,isHome,isWin,game_id,game_number,day_night,era,game,id,mlb_player_id,fullName,firstName,lastName,birthDate,currentAge,birthCity,birthStateProvince,birthCountry,height,weight,primaryPosition,batSide,pitchHand,debutDate,active,mlb_player_id,player_name,injury_date,season,injury_description,team_name
0,457759,season,fielding,2020,None,119,Los Angeles Dodgers,NaN,None,NA,32,32,32.0,63.0,17.0,4.0,84.0,.952,7.0,0.0,3.0,2.50,2.74,263.2,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,996,457759,Justin Turner,Justin,Turner,1984-11-23,40,Long Beach,CA,USA,"5' 11""",208,First Base,Right,Right,2009-09-08,1,457759,Justin Turner,2009-06-11,2009,Norfolk Tides placed 2B Justin Turner on the 7...,Norfolk Tides
1,457759,season,fielding,2020,None,119,Los Angeles Dodgers,NaN,None,NA,32,32,32.0,63.0,17.0,4.0,84.0,.952,7.0,0.0,3.0,2.50,2.74,263.2,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,996,457759,Justin Turner,Justin,Turner,1984-11-23,40,Long Beach,CA,USA,"5' 11""",208,First Base,Right,Right,2009-09-08,1,457759,Justin Turner,2012-05-29,2012,New York Mets placed 2B Justin Turner on the 1...,New York Mets
2,457759,season,fielding,2020,None,119,Los Angeles Dodgers,NaN,None,NA,32,32,32.0,63.0,17.0,4.0,84.0,.952,7.0,0.0,3.0,2.50,2.74,263.2,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,996,457759,Justin Turner,Justin,Turner,1984-11-23,40,Long Beach,CA,USA,"5' 11""",208,First Base,Right,Right,2009-09-08,1,457759,Justin Turner,2012-06-16,2012,New York Mets activated 1B Justin Turner from ...,New York Mets
3,457759,season,fielding,2020,None,119,Los Angeles Dodgers,NaN,None,NA,32,32,32.0,63.0,17.0,4.0,84.0,.952,7.0,0.0,3.0,2.50,2.74,263.2,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,996,457759,Justin Turner,Justin,Turner,1984-11-23,40,Long Beach,CA,USA,"5' 11""",208,First Base,Right,Right,2009-09-08,1,457759,Justin Turner,2013-06-18,2013,New York Mets placed 1B Justin Turner on the 1...,New York Mets
4,457759,season,fielding,2020,None,119,Los Angeles Dodgers,NaN,None,NA,32,32,32.0,63.0,17.0,4.0,84.0,.952,7.0,0.0,3.0,2.50,2.74,263.2,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,na,na,996,457759,Justin Turner,Justin,Turner,1984-11-23,40,Long Beach,CA,USA,"5' 11""",208,First Base,Right,Right,2009-09-08,1,457759,Justin Turner,2013-07-22,2013,New York Mets activated 3B Justin Turner from ...,New York Mets
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [92]:
# The Top 10 players with the most injuries

query = """
SELECT 
    player_name,
    COUNT(*) AS injury_count
FROM 
    injury_stats
GROUP BY 
    player_name
ORDER BY 
    injury_count DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)

,player_name,injury_count
0,Byron Buxton,37
1,Jason Heyward,35
2,Zach Eflin,31
3,Tyler O'Neill,31
4,Steven Matz,30
5,Jorge Soler,29
6,Max Fried,28
7,Lance McCullers Jr.,28
8,Mike Clevinger,27
9,Kyle Higashioka,27
